---
## Step 1: System Architecture

### RAG Pipeline Flow

![RAG pipleine](images/10_Building-a-Complete-RAG-Application.png)


### What Gets Tracked

**Experiment Level:**
- Model configuration
- Prompt versions
- Hyperparameters (top_k, temperature)

**Run Level:**
- Individual queries
- Performance metrics
- Cost per query

**Trace Level:**
- Every operation's timing
- Inputs and outputs
- Custom attributes

# Tutorial 1.9: Complete RAG Application

## Building a Production-Ready RAG System with Full Observability

Welcome to the final notebook! This brings together everything you've learned to build a complete, production-ready RAG (Retrieval-Augmented Generation) application with comprehensive MLflow tracking and tracing.

### What You'll Build

A full RAG system that includes:
- ✅ Document embedding and indexing
- ✅ Semantic search / retrieval
- ✅ Context-aware response generation
- ✅ Complete experiment tracking
- ✅ End-to-end tracing
- ✅ Cost monitoring
- ✅ Performance metrics
- ✅ Error handling
- ✅ Caching strategies
- ✅ RAG evaluation with RAGAS metrics

### Prerequisites
- Completed all previous notebooks (1.1-1.8)
- Understanding of experiment tracking and tracing

### Estimated Time: 25-30 minutes

---
## Step 2: Environment Setup

In [1]:
import time
import hashlib
from typing import List, Dict 
from datetime import datetime
from utils.clnt_utils import is_databricks_ai_gateway_client, get_databricks_ai_gateway_client, get_ai_gateway_model_names, get_openai_client

import numpy as np
import mlflow
from dotenv import load_dotenv

# Load environment
load_dotenv()

EXPERIMENT_NAME = "11-complete-rag-system"

# Configure MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(EXPERIMENT_NAME)

# Check if we are using a Databricks AI Gateway client
use_databricks_provider = is_databricks_ai_gateway_client()
if use_databricks_provider:
    client = get_databricks_ai_gateway_client()
    models = get_ai_gateway_model_names()
    JUDGE_MODEL = models[2]
    AGENT_MODEL = models[0]
    JUDGE_MODEL_URI = f"databricks:/{JUDGE_MODEL}"
else:
    # Initialize as an OpenAI client
    client = get_openai_client()
    JUDGE_MODEL = "gpt-5.2"
    AGENT_MODEL = "gpt-5.2"
    JUDGE_MODEL_URI = f"openai:/{JUDGE_MODEL}"

# Enable autologging
mlflow.openai.autolog()

print("✅ Environment configured for production RAG system")
print(f"   MLflow tracking: {mlflow.get_tracking_uri()}")
print(f"   Experiment: {EXPERIMENT_NAME}")
print(f"   Model: {AGENT_MODEL}")
print(f"   Judge Model: {JUDGE_MODEL}")
print(f"   Use Databricks Provider: {use_databricks_provider}")

2026/02/08 14:14:00 INFO mlflow.tracking.fluent: Experiment with name '11-complete-rag-system' does not exist. Creating a new experiment.


✅ Environment configured for production RAG system
   MLflow tracking: http://localhost:5000
   Experiment: 11-complete-rag-system
   Model: gpt-5.2
   Judge Model: gpt-5.2
   Use Databricks Provider: False


---
## Step 3: Document Store and Embeddings

In production, you'd use a vector database like Pinecone, Weaviate, or ChromaDB.
For this tutorial, we'll use in-memory storage.

In [2]:
# Sample document corpus (in production, load from database)
DOCUMENT_STORE = {  
    "doc1": "MLflow is an open source developer platform to build AI applications and models with confidence.",
    "doc2": "MLflow Tracing provides comprehensive observability for GenAI applications. It captures LLM calls, retrieval steps, tool usage, and agent reasoning with full input/output visibility.",
    "doc3": "MLflow integrates with 40+ frameworks including OpenAI, Anthropic, LangChain, LlamaIndex, DSPy, and AutoGen. Each integration provides automatic tracing and experiment tracking.",
    "doc4": "MLflow Evaluation enables systematic testing of GenAI applications using LLM-as-judge metrics, custom scorers, and human feedback. It supports both batch and online evaluation.",
    "doc5": "MLflow Prompt Registry allows teams to version, share, and manage prompts centrally. It tracks which prompts are used in which experiments and enables A/B testing.",
    "doc6": "MLflow supports collaborative development with experiment sharing, model versioning, and deployment tracking. Teams can compare results and iterate systematically.",
    "doc7": "MLflow provides cost tracking for LLM applications by monitoring token usage, API calls, and compute resources. This helps teams optimize spending and budget effectively.",
    "doc8": "MLflow is fully open source and vendor-neutral, ensuring no lock-in. It works with any cloud provider, ML framework, or LLM provider.",
    "doc9": "MLflow is a platform for the complete machine learning lifecycle. It provides experiment tracking, model packaging, and deployment capabilities across various ML frameworks.",
    "doc10": "MLflow offers Helpful Assitant, performaance metrics dashboards, Judge Builder online and continous monitoring, and MemAling for dyanmic optization, and more."
}
print(f"📚 Document store initialized with {len(DOCUMENT_STORE)} documents")

📚 Document store initialized with 10 documents


In [3]:
# Memory Cache for embeddings but in production, use Redis or something similar vector database
EMBEDDING_CACHE = {}

# creating a span for the embedding function
@mlflow.trace(name="embed_text", span_type="EMBEDDING")
def embed_text(text: str) -> List[float]:
    """
    Generate embeddings with caching.
    """
    # Check cache
    cache_key = hashlib.md5(text.encode()).hexdigest()
    span = mlflow.get_current_active_span()
    if cache_key in EMBEDDING_CACHE:
        span.set_attributes({"cache_hit": True})
        return EMBEDDING_CACHE[cache_key]

    span.set_attributes({"cache_hit": False, "text_length": len(text)})
    
    # Generate embedding
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    
    embedding = response.data[0].embedding
    span.set_attributes({"embedding_dim": len(embedding)})
    
    # Cache for future use
    EMBEDDING_CACHE[cache_key] = embedding
    
    return embedding

print("✅ Embedding function defined with caching")

✅ Embedding function defined with caching


### Compute embeedings for each doc and add them to the memory cache.

In [4]:
# Pre-compute document embeddings
print("\n🔄 Computing document embeddings...")

DOC_EMBEDDINGS = {}

for doc_id, text in DOCUMENT_STORE.items():
    DOC_EMBEDDINGS[doc_id] = embed_text(text)
    print(f"  ✓ {doc_id}")

print(f"\n✅ {len(DOC_EMBEDDINGS)} documents embedded and cached")


🔄 Computing document embeddings...
  ✓ doc1
  ✓ doc2
  ✓ doc3
  ✓ doc4
  ✓ doc5
  ✓ doc6
  ✓ doc7
  ✓ doc8
  ✓ doc9
  ✓ doc10

✅ 10 documents embedded and cached


[Trace(trace_id=tr-c106b163bd188919d16e3a8933926a32), Trace(trace_id=tr-8cd448de19b1842d09d611ffc2abcf21), Trace(trace_id=tr-1ae6f0970ca2197fdbc60c956a25ea35), Trace(trace_id=tr-e77a2ad4ecf83168a725a58c08849df3), Trace(trace_id=tr-3e4bb4f364595b3e3a398481403f5316), Trace(trace_id=tr-680d8ea9abd63ef6df4dfd766e29a643), Trace(trace_id=tr-e5950a65a4eae91e3e97fd0420ab73ea), Trace(trace_id=tr-59c83a8e98b1b0477d49452979f2ceab), Trace(trace_id=tr-b69b79544257a7879dc0272509c92ffd), Trace(trace_id=tr-9eed4aeb79ba445768e6abd7345f7cf1)]

---
## Step 4: Query Processing and Validation

In [5]:
@mlflow.trace(name="validate_query", span_type="PARSER")
def validate_query(query: str) -> Dict:
    """
    Validate and preprocess user query.
    """
    span = mlflow.get_current_active_span()
    span.set_attributes({"original_length": len(query)})
    
    # Basic validation
    if not query or len(query.strip()) == 0:
        mlflow.log_span_attribute("validation_error", "empty_query")
        raise ValueError("Query cannot be empty")
    
    if len(query) > 1000:
        span.set_attributes({"validation_warning": "query_too_long"})
        query = query[:1000]  # Truncate
    
    # Preprocess
    processed_query = query.strip()
    span.set_attributes({
        "processed_length": len(processed_query),
        "validation_passed": True
    })
    
    return {
        "original": query,
        "processed": processed_query,
        "valid": True
    }

print("✅ Query validation function defined")

✅ Query validation function defined


---
## Step 5: Semantic Search with Similarity Scoring

In [6]:
@mlflow.trace(name="semantic_search", span_type="RETRIEVER")
def search_documents(
    query_embedding: List[float],
    doc_embeddings: Dict[str, List[float]],
    top_k: int = 3,
    min_score: float = 0.7
) -> List[Dict]:
    """
    Search for most relevant documents using cosine similarity.
    """
    span = mlflow.get_current_active_span()
    span.set_attributes({
        "corpus_size": len(doc_embeddings),
        "top_k": top_k,
        "min_score": min_score
    })
    
    # Calculate cosine similarity for all documents
    scores = {}
    for doc_id, doc_emb in doc_embeddings.items():
        similarity = np.dot(query_embedding, doc_emb) / \
                     (np.linalg.norm(query_embedding) * np.linalg.norm(doc_emb))
        scores[doc_id] = float(similarity)
    
    # Sort by score and filter by minimum threshold
    sorted_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    filtered_docs = [(doc_id, score) for doc_id, score in sorted_docs if score >= min_score]
    
    # Get top k
    top_docs = filtered_docs[:top_k]
    
    # Return in LangChain Document format (page_content + metadata)
    # MLflow's RAGAS integration expects this format in RETRIEVER span outputs
    results = [
        {
            "page_content": DOCUMENT_STORE[doc_id],
            "metadata": {"doc_id": doc_id, "score": score}
        }
        for doc_id, score in top_docs
    ]
    
    # Log metrics
    if results:
        span.set_attributes({
            "num_results": len(results),
            "top_score": results[0]["metadata"]["score"],
            "avg_score": np.mean([r["metadata"]["score"] for r in results]),
            "min_result_score": results[-1]["metadata"]["score"]
        })
    else:
        span.set_attributes({"num_results": 0,
                             "retrieval_warning": "no_docs_above_threshold"})
    
    return results

print("✅ Semantic search function defined")

✅ Semantic search function defined


---
## Step 6: Context Assembly and Prompt Construction

In [7]:
@mlflow.trace(name="assemble_context", span_type="PARSER")
def assemble_context(query: str, retrieved_docs: List[Dict]) -> str:
    """
    Assemble context from retrieved documents and construct prompt.
    """
    span = mlflow.get_current_active_span()
    span.set_attributes({"num_docs": len(retrieved_docs)})
    
    if not retrieved_docs:
        span.set_attributes({"context_warning": "no_docs_retrieved"})
        return None
    
    # Format context
    context_parts = []
    for i, doc in enumerate(retrieved_docs, 1):
        context_parts.append(
            f"[Document {i}] (Relevance: {doc['metadata']['score']:.2f})\n{doc['page_content']}"
        )
    
    context = "\n\n".join(context_parts)
    
    # Construct prompt with system instructions
    prompt = f"""You are a helpful AI assistant that answers questions based on provided context.

INSTRUCTIONS:
- Answer the question using ONLY the information in the context below
- If the answer is not in the context, say "I don't have enough information to answer that"
- Be concise but complete
- Cite which document(s) you used if relevant

CONTEXT:
{context}

QUESTION: {query}

ANSWER:"""
    span.set_attributes({
        "context_length": len(context),
        "prompt_length": len(prompt)
    })
    
    return prompt

print("✅ Context assembly function defined")

✅ Context assembly function defined


---
## Step 7: LLM Response Generation

In [8]:
@mlflow.trace(name="generate_response", span_type="LLM")
def generate_answer(
    prompt: str,
    model: str = "gpt-5.2",
    temperature: float = 0.1,
) -> Dict:
    """
    Generate answer using LLM.
    """
    span = mlflow.get_current_active_span()
    span.set_attributes({
        "model": model,
        "temperature": temperature
    })
    
    if not prompt:
        span.set_attributes({"generation_error": "empty_prompt"})
        raise ValueError("Prompt cannot be empty")
    
    # Call LLM (automatically traced by OpenAI autolog)
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
    )
    
    answer = response.choices[0].message.content
    
    # Log generation metrics
    span.set_attributes({
        "answer_length": len(answer),
        "prompt_tokens": response.usage.prompt_tokens,
        "completion_tokens": response.usage.completion_tokens,
        "total_tokens": response.usage.total_tokens,
        "finish_reason": response.choices[0].finish_reason
    })
    span.set_attributes({
        "finish_reason": response.choices[0].finish_reason
    })
    
    return {
        "answer": answer,
        "tokens": response.usage.total_tokens,
        "model": model,
        "finish_reason": response.choices[0].finish_reason
    }

print("✅ Answer generation function defined")

✅ Answer generation function defined


---
## Step 8: Response Validation and Post-Processing

In [9]:
@mlflow.trace(name="validate_response", span_type="PARSER")
def validate_response(answer: str, min_length: int = 10) -> Dict:
    """
    Validate and post-process generated response.
    """
    span = mlflow.get_current_active_span()
    span.set_attributes({
        "min_length_threshold": min_length,
        "answer_length": len(answer)
    })
    
    issues = []
    
    # Check length
    if len(answer) < min_length:
        issues.append("too_short")
    
    # Check for common error patterns
    error_patterns = [
        "I don't have enough information",
        "I cannot answer",
        "I apologize"
    ]
    
    for pattern in error_patterns:
        if pattern.lower() in answer.lower():
            issues.append(f"contains_{pattern.replace(' ', '_').lower()}")
    
    span.set_attributes({
        "validation_issues": ",".join(issues) if issues else "none",
        "is_valid": len(issues) == 0
    })
    
    return {
        "is_valid": len(issues) == 0,
        "issues": issues,
        "answer": answer
    }

print("✅ Response validation function defined")

✅ Response validation function defined


---
## Step 9: Complete RAG Pipeline

In [10]:
@mlflow.trace(name="rag_pipeline", span_type="CHAIN")
def rag_qa_system(
    user_query: str,
    top_k: int = 3,
    min_score: float = 0.7,
    model: str = "gpt-5.2",
    temperature: float = 0.1
) -> Dict:
    """
    Complete RAG pipeline with full observability.
    
    Returns:
        Dict with answer, metadata, and status
    """
    # Start MLflow run for tracking
    with mlflow.start_run(run_name="rag_query"):
        
        # Log parameters
        mlflow.log_params({
            "query": user_query[:100],  # Truncate for readability
            "top_k": top_k,
            "min_score": min_score,
            "model": model,
            "temperature": temperature,
            "timestamp": datetime.now().isoformat()
        })
        
        try:
            # Step 1: Validate query
            validation_result = validate_query(user_query)
            processed_query = validation_result["processed"]
            
            # Step 2: Generate query embedding
            query_embedding = embed_text(processed_query)
            
            # Step 3: Search documents
            retrieved_docs = search_documents(
                query_embedding,
                DOC_EMBEDDINGS,
                top_k=top_k,
                min_score=min_score
            )
            
            if not retrieved_docs:
                mlflow.log_metric("retrieval_success", 0)
                return {
                    "status": "no_relevant_docs",
                    "answer": "I couldn't find relevant information to answer your question.",
                    "query": user_query,
                    "retrieved_docs": []
                }
            
            mlflow.log_metric("num_docs_retrieved", len(retrieved_docs))
            mlflow.log_metric("avg_relevance_score", 
                            np.mean([d["metadata"]["score"] for d in retrieved_docs]))
            mlflow.log_metric("retrieval_success", 1)
            
            # Step 4: Assemble context and construct prompt
            prompt = assemble_context(processed_query, retrieved_docs)
            
            # Step 5: Generate answer
            generation_result = generate_answer(
                prompt,
                model=model,
                temperature=temperature
            )
            
            # Step 6: Validate response
            validation = validate_response(generation_result["answer"])
            
            # Log final metrics
            mlflow.log_metric("total_tokens", generation_result["tokens"])
            mlflow.log_metric("answer_valid", 1 if validation["is_valid"] else 0)
            mlflow.log_metric("answer_length", len(generation_result["answer"]))
            
            # Log artifacts
            mlflow.log_text(user_query, "query.txt")
            mlflow.log_text(generation_result["answer"], "answer.txt")
            mlflow.log_text(prompt, "full_prompt.txt")
            mlflow.log_dict(
                {"docs": retrieved_docs},
                "retrieved_docs.json"
            )
            
            # Construct result
            result = {
                "status": "success",
                "query": user_query,
                "answer": generation_result["answer"],
                "retrieved_docs": retrieved_docs,
                "metadata": {
                    "num_docs": len(retrieved_docs),
                    "avg_relevance": float(np.mean([d["metadata"]["score"] for d in retrieved_docs])),
                    "tokens_used": generation_result["tokens"],
                    "model": model,
                    "is_valid": validation["is_valid"],
                    "validation_issues": validation["issues"]
                }
            }
            
            return result
            
        except Exception as e:
            # Log error
            mlflow.log_param("error_type", type(e).__name__)
            mlflow.log_param("error_message", str(e))
            mlflow.log_metric("pipeline_success", 0)
            
            return {
                "status": "error",
                "query": user_query,
                "error": str(e),
                "error_type": type(e).__name__
            }

print("✅ Complete RAG pipeline defined")

✅ Complete RAG pipeline defined


---
## Step 10: Testing the RAG System

In [11]:
# Test queries
test_queries = [
    "What tracing capabilities does MLflow provide?",
    "How does MLflow help with cost tracking?",
    "Can MLflow integrate with LangChain?",
    "What is the purpose of MLflow Prompt Registry?",
]

print("\n🧪 Testing RAG System\n")
print("="*80)

results = []

for i, query in enumerate(test_queries, 1):
    print(f"\nQuery {i}: {query}")
    print("-"*80)
    
    start_time = time.time()
    result = rag_qa_system(query, top_k=3, min_score=0.7)
    latency = time.time() - start_time
    
    if result["status"] == "success":
        print(f"\nAnswer: {result['answer']}")
        print("\nMetadata:")
        print(f"  - Documents used: {result['metadata']['num_docs']}")
        print(f"  - Avg relevance: {result['metadata']['avg_relevance']:.3f}")
        print(f"  - Tokens: {result['metadata']['tokens_used']}")
        print(f"  - Latency: {latency:.2f}s")
        print(f"  - Valid: {result['metadata']['is_valid']}")
        
        results.append({
            "query": query,
            "success": True,
            "latency": latency,
            "tokens": result['metadata']['tokens_used']
        })
    else:
        print(f"\nError: {result.get('error', 'Unknown error')}")
        results.append({
            "query": query,
            "success": False,
            "latency": latency
        })

print("\n" + "="*80)
print("\n✅ All queries processed!")


🧪 Testing RAG System


Query 1: What tracing capabilities does MLflow provide?
--------------------------------------------------------------------------------
🏃 View run rag_query at: http://localhost:5000/#/experiments/1/runs/5e2bdbf983b6452caa084b4728ee7c4c
🧪 View experiment at: http://localhost:5000/#/experiments/1

Answer: MLflow Tracing provides observability for GenAI applications by capturing **LLM calls, retrieval steps, tool usage, and agent reasoning**, with **full input/output visibility**. (Document 1)

Metadata:
  - Documents used: 1
  - Avg relevance: 0.730
  - Tokens: 181
  - Latency: 1.64s
  - Valid: True

Query 2: How does MLflow help with cost tracking?
--------------------------------------------------------------------------------
🏃 View run rag_query at: http://localhost:5000/#/experiments/1/runs/5ec49cf25c7d4210aaaf9ace33b8c1a9
🧪 View experiment at: http://localhost:5000/#/experiments/1

Answer: MLflow helps with cost tracking for LLM applications by monitoring 

[Trace(trace_id=tr-f2719003f4351aa5bc302c6afee70816), Trace(trace_id=tr-41e7296cf58f9b7dac8154b12cb02fc3), Trace(trace_id=tr-9b9aac05ba976832e955e0feb41994d8), Trace(trace_id=tr-431f6318cb33f29e2b5e0fbc94428d6a)]

---
## Step 11: Performance Analysis

In [12]:
# Analyze performance
print("\n📊 Performance Summary\n")
print("="*60)

successful = [r for r in results if r["success"]]

if successful:
    latencies = [r["latency"] for r in successful]
    tokens = [r["tokens"] for r in successful]
    
    print(f"Success Rate: {len(successful)}/{len(results)} ({len(successful)/len(results)*100:.1f}%)")
    print("\nLatency Stats:")
    print(f"  Average: {np.mean(latencies):.2f}s")
    print(f"  Min: {np.min(latencies):.2f}s")
    print(f"  Max: {np.max(latencies):.2f}s")
    print(f"  Std Dev: {np.std(latencies):.2f}s")
    
    print("\nToken Usage:")
    print(f"  Average: {np.mean(tokens):.0f} tokens")
    print(f"  Total: {np.sum(tokens):.0f} tokens")
    print(f"  Est. Cost: ${np.sum(tokens) * 0.15 / 1_000_000:.6f}")
    
    print("\nCache Performance:")
    print(f"  Embedding cache hits: {len(EMBEDDING_CACHE)} embeddings cached")
    
print("\n" + "="*60)


📊 Performance Summary

Success Rate: 3/4 (75.0%)

Latency Stats:
  Average: 1.76s
  Min: 1.64s
  Max: 1.93s
  Std Dev: 0.13s

Token Usage:
  Average: 180 tokens
  Total: 540 tokens
  Est. Cost: $0.000081

Cache Performance:
  Embedding cache hits: 14 embeddings cached



---
## Step 12: RAG Evaluation with RAGAS Metrics

Now let's evaluate our RAG system using RAGAS (Retrieval-Augmented Generation Assessment) metrics. These metrics help assess the quality of both retrieval and generation.

| Metric | What It Measures | Requirements |
|--------|-----------------|--------------|
| **Faithfulness** | Is the answer grounded in the retrieved context? | Traces with RETRIEVER spans |
| **ContextRelevance** | Is the retrieved context relevant to the query? | Traces with RETRIEVER spans |
| **ContextPrecision** | Are retrieved docs ranked by relevance to expected output? | `expectations['expected_output']` |

**Important:** RAGAS scorers extract context from **traces with RETRIEVER spans**, not from static datasets. This is why we search for traces from our RAG pipeline rather than constructing a manual dataset.

In [13]:
# Workaround for async event loop issues in Jupyter notebooks
# Libraries like RAGAS run async code that can conflict with Jupyter's event loop

import logging

# 1. Use nest_asyncio to allow nested event loops (essential for Jupyter + async libraries)
import nest_asyncio
nest_asyncio.apply()

# 2. Suppress noisy asyncio error messages (they don't affect results)
logging.getLogger("asyncio").setLevel(logging.CRITICAL)

# 3. If litellm is installed, disable its async logging to prevent conflicts
try:
    import litellm
    litellm.success_callback = []
    litellm.failure_callback = []
    litellm._async_success_callback = []
    litellm._async_failure_callback = []
    litellm.disable_streaming_logging = True
    litellm.turn_off_message_logging = True
    logging.getLogger("LiteLLM").setLevel(logging.WARNING)
    print("   - LiteLLM async logging disabled")
except ImportError:
    pass  # litellm not installed, no workaround needed

print("✅ Jupyter async compatibility configured")
print("   - nest_asyncio applied for nested event loop support")
print("   - Asyncio error logging suppressed")

   - LiteLLM async logging disabled
✅ Jupyter async compatibility configured
   - nest_asyncio applied for nested event loop support
   - Asyncio error logging suppressed


In [14]:
from mlflow.genai.scorers.ragas import Faithfulness, ContextRelevance


# Initialize RAGAS scorers (requires: pip install ragas)
# Note: We're using Faithfulness and ContextRelevance which work with traces containing RETRIEVER spans
# ContextPrecision is not included because it requires expectations['expected_output']

faithfulness_scorer = Faithfulness(model=JUDGE_MODEL_URI)
context_relevance_scorer = ContextRelevance(model=JUDGE_MODEL_URI)

print("✅ RAGAS scorers initialized:")
print("   - Faithfulness (checks if answer is grounded in retrieved context)")
print("   - ContextRelevance (checks if retrieved context is relevant to query)")
print("\n⚠️  ContextPrecision not used - it requires expected_output in expectations")

✅ RAGAS scorers initialized:
   - Faithfulness (checks if answer is grounded in retrieved context)
   - ContextRelevance (checks if retrieved context is relevant to query)

⚠️  ContextPrecision not used - it requires expected_output in expectations


In [15]:
# Get traces from the RAG pipeline runs (which contain RETRIEVER spans)
# RAGAS scorers like Faithfulness and ContextRelevance extract context 
# from RETRIEVER spans in traces, not from static datasets

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

# Search for traces from our RAG pipeline, limited to the most recent batch
# (avoids picking up old traces that may have different output formats)
rag_traces = mlflow.search_traces(
    experiment_ids=[experiment.experiment_id],
    filter_string="name = 'rag_pipeline'",
    max_results=len(test_queries),
)

print(f"✅ Found {len(rag_traces)} RAG pipeline traces for evaluation")
print("\nThese traces contain RETRIEVER spans that RAGAS scorers need:")
print("   - semantic_search span with retrieved documents")
print("   - Full input/output flow for faithfulness checking")

# Verify traces have the required page_content field in RETRIEVER outputs
from mlflow.entities import Trace as TraceEntity
sample_trace = rag_traces.iloc[0]['trace']
if isinstance(sample_trace, str):
    sample_trace = TraceEntity.from_json(sample_trace)
retriever_spans = [s for s in sample_trace.data.spans if s.span_type == 'RETRIEVER']
if retriever_spans:
    outputs = retriever_spans[0].outputs
    if isinstance(outputs, list) and len(outputs) > 0 and isinstance(outputs[0], dict):
        has_page_content = 'page_content' in outputs[0]
        print(f"\n   Retriever output keys: {list(outputs[0].keys())}")
        print(f"   Has page_content: {has_page_content}")
        if not has_page_content:
            print("\n⚠️  WARNING: Traces missing 'page_content' in RETRIEVER outputs!")
            print("   Re-run from Step 5 to regenerate traces with the correct format.")

/var/folders/mg/tr3ld7vx6gq1ym7p48vjgmpc0000gp/T/ipykernel_95909/2815135260.py:9: FutureWarning: Parameter 'experiment_ids' is deprecated. Please use 'locations' instead.
  rag_traces = mlflow.search_traces(


✅ Found 4 RAG pipeline traces for evaluation

These traces contain RETRIEVER spans that RAGAS scorers need:
   - semantic_search span with retrieved documents
   - Full input/output flow for faithfulness checking

   Retriever output keys: ['page_content', 'metadata']
   Has page_content: True


### Run evaluation on the traces

In [16]:
# Run RAGAS evaluation on traces (which contain RETRIEVER spans)
# Note: ContextPrecision requires expected_output in expectations, so we skip it here
# To use ContextPrecision, you'd need to log expectations to traces:
#   mlflow.log_expectation(trace_id, name='expected_output', value='...', source='...')

print("🔄 Running RAGAS evaluation on RAG traces...\n")

ragas_results = mlflow.genai.evaluate(
    data=rag_traces,  # Pass traces (not static data) - they contain RETRIEVER spans
    scorers=[
        faithfulness_scorer,      # Checks if answer is grounded in retrieved context
        context_relevance_scorer, # Checks if retrieved context is relevant to query
    ]
)

print("\n✅ RAGAS evaluation complete!")
print("\n📊 RAGAS Metrics Summary:")
print("-" * 50)
for metric_name, value in ragas_results.metrics.items():
    if isinstance(value, float):
        print(f"  {metric_name}: {value:.3f}")
    else:
        print(f"  {metric_name}: {value}")

2026/02/08 14:15:21 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.


🔄 Running RAGAS evaluation on RAG traces...



Evaluating:   0%|          | 0/4 [Elapsed: 00:00, Remaining: ?] 

2026/02/08 14:15:21 ERROR mlflow.genai.scorers.ragas: Missing required parameter for RAGAS metric ContextRelevance: RAGAS metric 'ContextRelevance' requires 'trace with retrieval spans' parameter, which is missing. 
Make sure your trace includes retrieval spans. Example: use @mlflow.trace(span_type=SpanType.RETRIEVER) decorator
2026/02/08 14:15:21 ERROR mlflow.genai.scorers.ragas: Missing required parameter for RAGAS metric Faithfulness: RAGAS metric 'Faithfulness' requires 'trace with retrieval spans' parameter, which is missing. 
Make sure your trace includes retrieval spans. Example: use @mlflow.trace(span_type=SpanType.RETRIEVER) decorator
/Users/jules/git-repos/mlflow-misc/.venv/lib/python3.11/site-packages/litellm/litellm_core_utils/logging_worker.py:75: RuntimeWarning: coroutine 'Logging.async_success_handler' was never awaited
  self._queue = None



✅ RAGAS evaluation complete!

📊 RAGAS Metrics Summary:
--------------------------------------------------
  ContextRelevance/mean: 1.000
  Faithfulness/mean: 0.944


---
## Step 13: Viewing Results in MLflow UI

Now let's explore what was tracked in MLflow.

---
## Step 12: Viewing Results in MLflow UI

Now let's explore what was tracked in MLflow.

In [17]:
print("""
╔══════════════════════════════════════════════════════════════╗
║         Analyzing Results in MLflow UI                       ║
╚══════════════════════════════════════════════════════════════╝

🔍 EXPERIMENTS VIEW:
   Navigate to: http://localhost:5000
   Select: "10-complete-rag-system" experiment
   
   You'll see:
   - All RAG query runs
   - Parameters (query, model, top_k)
   - Metrics (tokens, relevance, latency)
   - Artifacts (queries, answers, prompts)

📊 COMPARING RUNS:
   1. Select multiple runs
   2. Click "Compare"
   3. View side-by-side:
      - Which queries used most tokens?
      - Which had highest relevance scores?
      - Performance variations

🌳 TRACES VIEW:
   Click "Traces" tab to see:
   
   Timeline visualization:
   rag_pipeline (CHAIN) ━━━━━━━━━━━━━━━━━━━━ 2.5s
   ├─ validate_query (PARSER) ━━ 0.01s
   ├─ embed_text (EMBEDDING) ━━━━ 0.3s
   ├─ semantic_search (RETRIEVER) ━ 0.05s
   ├─ assemble_context (PARSER) ━ 0.02s
   ├─ generate_response (LLM) ━━━━━ 2.0s
   │  └─ OpenAI API call ━━━━━━━━━ 1.9s
   └─ validate_response (PARSER) ━ 0.01s

🔎 SPAN DETAILS:
   Click on any span to see:
   - Inputs and outputs
   - Custom attributes
   - Timing information
   - Cache hit status
   - Relevance scores

📈 KEY INSIGHTS:
   1. Performance Bottlenecks:
      - Which step takes longest?
      - Is it the LLM or retrieval?
   
   2. Quality Metrics:
      - Average relevance scores
      - Documents per query
      - Answer validation rates
   
   3. Cost Analysis:
      - Token usage per query
      - Cache effectiveness
      - Cost per operation
   
   4. Error Patterns:
      - Failed queries
      - Low relevance scores
      - Validation issues

💡 OPTIMIZATION OPPORTUNITIES:
   Based on traces, you can:
   - Adjust top_k if retrieval is slow
   - Increase min_score if quality is poor
   - Optimize prompts to reduce tokens
   - Add more aggressive caching
   - Implement parallel retrieval

""")


╔══════════════════════════════════════════════════════════════╗
║         Analyzing Results in MLflow UI                       ║
╚══════════════════════════════════════════════════════════════╝

🔍 EXPERIMENTS VIEW:
   Navigate to: http://localhost:5000
   Select: "10-complete-rag-system" experiment

   You'll see:
   - All RAG query runs
   - Parameters (query, model, top_k)
   - Metrics (tokens, relevance, latency)
   - Artifacts (queries, answers, prompts)

📊 COMPARING RUNS:
   1. Select multiple runs
   2. Click "Compare"
   3. View side-by-side:
      - Which queries used most tokens?
      - Which had highest relevance scores?
      - Performance variations

🌳 TRACES VIEW:
   Click "Traces" tab to see:

   Timeline visualization:
   rag_pipeline (CHAIN) ━━━━━━━━━━━━━━━━━━━━ 2.5s
   ├─ validate_query (PARSER) ━━ 0.01s
   ├─ embed_text (EMBEDDING) ━━━━ 0.3s
   ├─ semantic_search (RETRIEVER) ━ 0.05s
   ├─ assemble_context (PARSER) ━ 0.02s
   ├─ generate_response (LLM) ━━━━━ 2.0s
  

---
## Summary

Congratulations! You've built a complete, production-ready RAG system with:

### ✅ Core Functionality
- Document embedding and indexing
- Semantic search with cosine similarity
- Context-aware response generation
- Input validation and output validation

### ✅ Observability
- Complete experiment tracking
- End-to-end distributed tracing
- Custom span attributes
- Performance metrics
- Cost tracking

### ✅ Production Features
- Embedding caching
- Error handling
- Query validation
- Response validation
- Relevance scoring

### ✅ RAG Evaluation
- RAGAS metrics (Faithfulness, Context Precision)
- Retrieval groundedness scoring
- LLM-as-Judge evaluation patterns

### 🎯 What You Can Do Next

**Immediate Improvements:**
1. Add a vector database (Pinecone, Weaviate, ChromaDB)
2. Implement streaming responses
3. Add reranking for better relevance
4. Create a web UI (Streamlit, Gradio)
5. Set up monitoring dashboards

**Production Enhancements:**
1. Deploy as an API (FastAPI)
2. Add rate limiting
3. Implement user authentication
4. Set up A/B testing framework
5. Create alerting for quality/cost

### 📚 Complete Tutorial Series

You've now completed Tutorial 1:

1. ✅ **Setup and Introduction**
2. ✅ **Experiment Tracking for LLMs**
3. ✅ **Introduction to Tracing**
4. ✅ **Manual Tracing and Advanced Observability**
5. ✅ **Prompt Management**
6. ✅ **Framework Integrations**
7. ✅ **Evaluating Agents**
8. ✅ **Prompt Optimization with GEPA**
9. ✅ **Complete RAG Application** (This notebook!)

### 🚀 Next Tutorial Series

Ready to go deeper?

- **Tutorial 2**: Prompt Engineering and Version Control
- **Tutorial 3**: Advanced Tracing and Debugging
- **Tutorial 4**: Advanced Agent Evaluation
- **Tutorial 5**: Optimizing Prompts for Performance

### 🎉 Congratulations!

You now have:
- A production-ready RAG system
- Complete observability into your GenAI app
- Understanding of MLflow's core capabilities
- Patterns for building scalable LLM applications
- RAG evaluation using RAGAS metrics

**Keep building, keep learning, and happy coding!** 🚀